<a href="https://colab.research.google.com/github/444112029012/4112029012_cal/blob/main/data_collection/html_dataset/%E5%89%B5%E5%BB%BA%E9%9D%9EAI%E6%AC%84%E4%BD%8DHTML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%capture
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib

# 改style要在改font之前
# plt.style.use('seaborn')

matplotlib.font_manager.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
matplotlib.rc('font', family='Taipei Sans TC Beta')
from google.colab import files
import pandas as pd

def load_data() -> pd.DataFrame:
  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  # 將上傳的檔案讀取到 pandas DataFrame 中
  # 假設上傳的檔案是 CSV 格式。如果是其他格式，您可能需要調整讀取函數（例如：pd.read_excel）
  file_name = next(iter(uploaded))
  df = pd.read_csv(file_name)

  # 顯示 DataFrame 的前 5 行
  display(df.head())
  return df
# df = load_data()
def save_df(df, filename = 'phishing_dataset_for_trainning'):
    """
    將指定的 DataFrame 存成 CSV 檔案。

    參數:
    - df: 要儲存的 DataFrame
    - filename: 存檔的檔名（例如 'output.csv'）

    功能:
    - 使用 UTF-8 with BOM 編碼避免中文亂碼
    - 不包含索引欄位
    """
    try:
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✅ 資料已成功儲存至 '{filename}'")
    except Exception as e:
        print(f"❌ 儲存失敗：{e}")

In [11]:
%%capture
# !pip install google-generativeai
!pip install selenium
# !pip install webdriver-manager
!apt-get update
# !apt install chromium-chromedriver
# 2. 手動安裝與最新 WebDriver 相符的 Chrome 瀏覽器
#    這能確保版本匹配，避免啟動錯誤
# !wget -q https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/114.0.5735.90/linux64/chrome-linux64.zip
# !unzip -q chrome-linux64.zip -d /bin/

In [9]:
%%capture
!pip install google-colab-selenium

In [ ]:
save_df(df_with_html_features, 'phishing_dataset_expansion_1_html_test.csv') # 記得!!!

✅ 資料已成功儲存至 'phishing_dataset_expansion_forEmbeddingModule_24000to36000.csv'


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import re
import time
from collections import Counter
import ipaddress
import signal
from tqdm import tqdm
from typing import Tuple, Optional
import os
import numpy as np

# ====== 可選: 如果需要處理動態內容，請取消註釋以下內容並安裝相關庫 ======
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
from google_colab_selenium import Chrome
# =====================================================================


def get_html_content(url, timeout=20, max_retries=2):
    """
    根據 URL 獲取網頁的完整 HTML 內容，並偵測是否有轉向。
    先使用 requests 爬取，如果內容為空或失敗則使用 Selenium。
    Args:
        url (str): 目標網頁的 URL。
        timeout (int): 請求超時時間 (秒)。
        max_retries (int): 失敗時重試次數。
    Returns:
        tuple[str, bool]: 網頁的 HTML 內容，如果獲取失敗則返回 None。
                           以及一個布林值，表示是否發生了轉向。
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.7258.66 Safari/537.36'
    }

    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout, verify=False)
            response.raise_for_status()
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')
            body_content = soup.find('body')
            response.close()
            # 檢查內容是否為空或過短（可能是動態內容）
            if html_content and len(html_content.strip()) > 100:
                # 檢查頁面內容是否包含 "page not found" 相關關鍵詞
                not_found_keywords = ['page not found', 'error 404', 'page does not exist', '找不到頁面', '頁面不存在']
                matched_keywords = 0
                try:
                  body_text = body_content.get_text(strip=True)
                  body_lower = body_text.lower()

                  # 計算匹配的關鍵詞數量
                  matched_keywords = sum(1 for keyword in not_found_keywords if keyword in body_lower)
                except:
                  html_lower = html_content.lower()

                  # 計算匹配的關鍵詞數量
                  matched_keywords = sum(1 for keyword in not_found_keywords if keyword in html_lower)
                if matched_keywords >= 2:
                    print(f"❌ URL {url} 頁面內容包含 {matched_keywords} 個 'page not found' 相關關鍵詞，直接跳過")
                    return None, False

                print(f"✅ 使用 requests 成功獲取 {url} 的內容")
                return html_content, False
            else:
                print(f"⚠️  requests 獲取的內容為空或過短，將嘗試使用 Selenium")
                break

        except requests.exceptions.HTTPError as e:
            # 檢查是否為 404 或其他 "page not found" 相關錯誤
            if response.status_code == 404:
                print(f"❌ URL {url} 返回 404 錯誤，直接跳過")
                return None, False
            elif response.status_code >= 400:
                print(f"❌ URL {url} 返回 HTTP {response.status_code} 錯誤，直接跳過")
                return None, False
            else:
                print(f"[重試 {attempt+1}/{max_retries}] Requests 獲取 {url} 時發生 HTTP 錯誤: {e}")
                if attempt == max_retries:
                    print(f"❌ Requests 重試 {max_retries} 次後仍然失敗，將嘗試使用 Selenium")
                    break
                time.sleep(0.5)
        except requests.exceptions.RequestException as e:
            print(f"[重試 {attempt+1}/{max_retries}] Requests 獲取 {url} 時發生錯誤: {e}")
            if attempt == max_retries:
                print(f"❌ Requests 重試 {max_retries} 次後仍然失敗，將嘗試使用 Selenium")
                break
            time.sleep(1)

    return None, None

def set_driver():
    print('開始設定DRIVER...')
    try:
        chrome_options = Options()
        chrome_options.add_argument('--headless') # 啟用無頭模式，讓瀏覽器在背景運行，不顯示視窗
        chrome_options.add_argument('--no-sandbox') # 避免在 Colab 環境下可能發生的權限問題
        chrome_options.add_argument('--disable-dev-shm-usage') # 解決 /dev/shm 分區空間不足的問題，這在 Docker 或 Colab 中很常見
        chrome_options.add_argument('--disable-gpu') # 禁用 GPU 加速
        chrome_options.add_argument('--disable-setuid-sandbox') # 讓 Chrome 可以在不安全的環境下執行

        # 模擬一個真實的 User-Agent，避免被網站偵測為爬蟲
        chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"')
        prefs = {
        "download.default_directory": "NUL",      # (如果是 Windows, 用這個)
        "download.prompt_for_download": False,      # 不詢問下載位置
        "download.directory_upgrade": True,
        "profile.default_content_settings.popups": 0, # 封鎖彈出視窗
        "safebrowsing.enabled": False,                # 關閉安全瀏覽 (避免它干擾爬蟲)
        "profile.default_content_setting_values.automatic_downloads": 2 # *** 關鍵：禁止自動下載 ***
        }
        chrome_options.add_experimental_option("prefs", prefs)
        driver = Chrome(options=chrome_options)
        driver.set_page_load_timeout(50)
        driver.set_script_timeout(50)
        print("WebDriver 初始化完成。")
        return driver
    except Exception as e:
        print('driver設定錯誤:', e)

def _fetch_dynamic_content(driver, url: str) -> Tuple[Optional[str], str]:
    """
    使用 Selenium 爬取動態網頁內容。
    """
    try:


        driver.get(url)
        # 等待網頁內容載入，可依據網站特性調整時間
        time.sleep(3.5)
        for i in range(1, 3):
            driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight * {i/3});")
            time.sleep(0.8)

        print(f"✅ 網頁 {url} 動態爬取完成...")
        text = driver.page_source
        return (text, 'OK_Dynamic') if text else (None, 'OK_Dynamic_Empty')
    except Exception as e:
        print(f"❌ 錯誤: 使用 Selenium 爬取網頁時發生錯誤。訊息: {e}")
        return None, 'Error_Selenium'


def extract_html_features(df: pd.DataFrame, url_column: str = 'url') -> pd.DataFrame:

    if url_column not in df.columns:
        raise ValueError(f"DataFrame 中未找到指定的 URL 欄位: '{url_column}'")

    # 確保所有 URL 都是字串類型，並處理可能的 NaN
    df[url_column] = df[url_column].astype(str).replace('nan', '')

    # 更新特徵欄位列表，加入兩個轉向特徵
    html_feature_columns = [
        'phish_hints', 'domain_in_brand', 'nb_hyperlinks', 'ratio_intHyperlinks',
        'ratio_extHyperlinks', 'ratio_extRedirection', 'ratio_extErrors',
        'external_favicon', 'links_in_tags', 'ratio_extMedia', 'safe_anchor',
        'empty_title', 'domain_in_title', 'domain_with_copyright',
        'has_meta_refresh', 'has_js_redirect',
        'feature_extracted'  # 新增欄位
    ]

    results = []
    driver = set_driver()
    RECYCLE_DRIVER_COUNT = 100
    if 'feature_extracted' not in df.columns:
        df[[html_feature_columns]] = np.nan
    try:
        for index, row in df.iterrows():
            if len(results) % RECYCLE_DRIVER_COUNT == 0 and results:
                driver.quit()
                driver = set_driver()
            url = row[url_column]
            if row['feature_extracted'] == 0 or row['feature_extracted'] == 1:
                continue
            if not url:
                df.at[index, 'feature_extracted'] = 0.0
                continue
            # if row['feature_extracted'] == 1:
            #     for col in html_feature_columns:
            #         features[col] = row[col]
            #     results.append(features)
            #     continue
            for col in html_feature_columns:
                df.at[index, col] = 0.0
            print(f"正在處理第 {index+1} / {len(df)} 筆 URL: {url}")

            # 設定單個 URL 的最大處理時間（秒）
            max_url_time = 30

            try:
                # --- 關鍵修改：呼叫 get_html_content 並接收兩個返回值 ---
                html_content, dynamic_content = safe_process_url(driver, url, max_time=120)
            except Exception as e:
                print(f"❌ 處理 URL {url} 時發生未預期錯誤: {e}")
                df.at[index, 'feature_extracted'] = 0.0
                continue

            if html_content and dynamic_content:
                try:
                    soup_static = BeautifulSoup(html_content, 'html.parser')
                    parsed_url = urlparse(url)
                    base_domain = parsed_url.netloc.split(':')[0]
                    if base_domain.startswith('www.'):
                        base_domain = base_domain[4:]
                    redirect_keywords = [
                        "window.location.href",
                        "location.href",
                        "location.assign",
                        "location.replace",
                        "window.navigate"
                    ]
                    # --- 新增的偵測邏輯 ---
                    # 15. 偵測 meta 轉向
                    meta_refresh_tag = soup_static.find('meta', attrs={'http-equiv': lambda x: x and x.lower() == 'refresh'})
                    if meta_refresh_tag:
                        content = meta_refresh_tag.get("content", "")
                        df.at[index, 'has_meta_refresh'] = 1.0 if "url=" in content.lower() else 0.0  # 確認是 redirect，不是單純 reload
                    else:
                        df.at[index, 'has_meta_refresh'] = 0.0

                    # 16. 偵測 JavaScript 轉向 (來自 get_html_content 的返回值)
                    df.at[index, 'has_js_redirect'] = 1.0 if soup_static.find("script", string=lambda s: any(k in s for k in redirect_keywords) if s else False) else 0.0
                    # ----------------------
                    soup_dynamic = BeautifulSoup(dynamic_content, 'html.parser')

                    # --- 1. phish_hints: HTML 內容中是否存在常見的釣魚提示詞語 ---
                    phish_keywords = ['login', 'signin', 'account update', 'verify account',
                                    'security alert', 'password', 'bank', 'paypal', 'credit card',
                                    '緊急', '驗證', '登入', '帳戶更新', '安全警告', '密碼']
                    text_content = soup_dynamic.get_text().lower()
                    df.at[index, 'phish_hints'] = 1 if any(kw in text_content for kw in phish_keywords) else 0.0

                    # --- 2. domain_in_brand: 網站內容中提及的品牌名稱是否與域名一致 ---
                    # 簡化處理: 檢查域名核心部分是否出現在 meta description, title 或 copyright 中
                    # 更精確需要品牌名稱列表或 NLP 實體識別
                    brand_match = 0
                    domain_parts = base_domain.split('.')
                    # 改進域名提取邏輯：處理多級域名
                    if len(domain_parts) >= 2:
                        # 對於常見的頂級域名，取倒數第二個部分
                        tld = domain_parts[-1]
                        if tld in ['com', 'org', 'net', 'edu', 'gov', 'mil']:
                            core_domain = domain_parts[-2]
                        else:
                            # 對於其他域名，取倒數第三個部分（如果存在）
                            core_domain = domain_parts[-3] if len(domain_parts) >= 3 else domain_parts[-2]
                    else:
                        core_domain = domain_parts[0]

                    title_tag = soup_dynamic.find('title')
                    if title_tag and core_domain in title_tag.get_text().lower():
                        brand_match = 1
                    elif soup_dynamic.find('meta', attrs={'name': 'description'}) and core_domain in soup_dynamic.find('meta', attrs={'name': 'description'})['content'].lower():
                        brand_match = 1
                    df.at[index, 'domain_in_brand'] = brand_match


                    # --- 3. nb_hyperlinks: 網頁中超連結的總數 ---
                    all_links = soup_dynamic.find_all('a', href=True)
                    df.at[index, 'nb_hyperlinks'] = len(all_links)

                    # --- 4. ratio_intHyperlinks: 內部超連結的比例 ---
                    # --- 5. ratio_extHyperlinks: 外部超連結的比例 ---
                    internal_links = 0
                    external_links = 0
                    for link_tag in all_links:
                        href = link_tag['href']
                        full_url = urljoin(url, href)
                        linked_domain = urlparse(full_url).netloc
                        if linked_domain == parsed_url.netloc:
                            internal_links += 1
                        else:
                            external_links += 1
                    total_links_calc = internal_links + external_links
                    df.at[index, 'ratio_intHyperlinks'] = internal_links / total_links_calc if total_links_calc > 0 else 0.0
                    df.at[index, 'ratio_extHyperlinks'] = external_links / total_links_calc if total_links_calc > 0 else 0.0


                    # --- 6. ratio_extRedirection (外部重新導向的比例) ---
                    # 檢查外部連結是否包含重定向相關的屬性或JavaScript
                    redirect_count = 0
                    for link_tag in all_links:
                        href = link_tag['href']
                        if href.startswith('#'):
                            continue
                        full_url = urljoin(url, href)
                        linked_parsed = urlparse(full_url)
                        linked_domain = linked_parsed.netloc
                        is_external = (linked_domain != parsed_url.netloc)

                        if is_external:
                            # 檢查是否有重定向相關的屬性
                            if link_tag.get('onclick') and 'window.location' in link_tag.get('onclick', ''):
                                redirect_count += 1
                            elif link_tag.get('target') == '_blank' and 'redirect' in link_tag.get_text().lower():
                                redirect_count += 1

                    df.at[index, 'ratio_extRedirection'] = redirect_count / len(all_links) if all_links else 0.0

                    # --- 7. ratio_extErrors (外部連結中返回錯誤的比例) ---
                    # 檢查外部連結是否指向明顯錯誤的URL格式
                    error_count = 0
                    for link_tag in all_links:
                        href = link_tag['href']
                        if href.startswith('#'):
                            continue
                        full_url = urljoin(url, href)
                        linked_parsed = urlparse(full_url)
                        linked_domain = linked_parsed.netloc
                        is_external = (linked_domain != parsed_url.netloc)

                        if is_external:
                            # 檢查是否為明顯錯誤的URL
                            if 'error' in full_url.lower() or '404' in full_url or 'notfound' in full_url.lower():
                                error_count += 1
                            elif not linked_domain or linked_domain == '':
                                error_count += 1

                    df.at[index, 'ratio_extErrors'] = error_count / len(all_links) if all_links else 0.0

                    # --- 8. external_favicon: 網站是否使用來自外部域名的 Favicon ---
                    favicon_link = soup_dynamic.find('link', rel=lambda x: x and 'icon' in x.lower())
                    df.at[index, 'external_favicon'] = 0.0
                    if favicon_link and 'href' in favicon_link.attrs:
                        favicon_url = urljoin(url, favicon_link['href'])
                        favicon_domain = urlparse(favicon_url).netloc
                        if favicon_domain != parsed_url.netloc:
                            df.at[index, 'external_favicon'] = 1.0

                    # --- 9. links_in_tags: 特定 HTML 標籤（如 <a>、<script>）中連結的數量 ---
                    # 這裡統計所有 href 和 src 屬性的連結
                    total_links_in_tags = 0
                    for tag in soup_dynamic.find_all(['a', 'script', 'img', 'link', 'iframe', 'form']):
                        if 'href' in tag.attrs:
                            total_links_in_tags += 1
                        if 'src' in tag.attrs:
                            total_links_in_tags += 1
                        if tag.name == 'form' and 'action' in tag.attrs:
                            total_links_in_tags += 1
                    df.at[index, 'links_in_tags'] = total_links_in_tags

                    # --- 10. ratio_extMedia: 外部媒體（圖片、音頻、視頻）的比例 ---
                    media_tags = soup_dynamic.find_all(['img', 'audio', 'video', 'source'])
                    total_media = len(media_tags)
                    external_media = 0
                    for media_tag in media_tags:
                        src = media_tag.get('src') or media_tag.get('href')
                        if src:
                            media_url = urljoin(url, src)
                            media_domain = urlparse(media_url).netloc
                            if media_domain != parsed_url.netloc:
                                external_media += 1
                    df.at[index, 'ratio_extMedia'] = external_media / total_media if total_media > 0 else 0.0

                    # --- 11. safe_anchor: 錨點連結是否安全（例如避免指向可疑外部網站） ---
                    # 檢查是否有指向 IP 地址、不常見協議或可疑域名的外部連結
                    df.at[index, 'safe_anchor'] = 1.0
                    suspicious_keywords = ['bit.ly', 'tinyurl', 'goo.gl', 't.co', 'fb.me', 'is.gd']

                    for link_tag in all_links:
                        href = link_tag['href']
                        if href.startswith('#'):
                            continue
                        full_url = urljoin(url, href)
                        linked_parsed = urlparse(full_url)
                        linked_domain = linked_parsed.netloc
                        is_external = (linked_domain != parsed_url.netloc)

                        if is_external:
                            # 檢查是否為IP地址
                            try:
                                ipaddress.ip_address(linked_domain)
                                df.at[index, 'safe_anchor'] = 0.0
                                break
                            except ValueError:
                                pass

                            # 檢查協議是否安全
                            if linked_parsed.scheme not in ['http', 'https', '']:
                                df.at[index, 'safe_anchor'] = 0.0
                                break

                            # 檢查是否為可疑的短網址服務
                            if any(keyword in linked_domain.lower() for keyword in suspicious_keywords):
                                df.at[index, 'safe_anchor'] = 0.0
                                break

                    # --- 12. empty_title: 網頁標題是否為空 ---
                    df.at[index, 'empty_title'] = 1.0 if not (soup_dynamic.title and soup_dynamic.title.string and soup_dynamic.title.string.strip()) else 0.0

                    # --- 13. domain_in_title: 域名是否出現在網頁標題中 ---
                    df.at[index, 'domain_in_title'] = 0.0
                    if soup_dynamic.title and soup_dynamic.title.string:
                        if base_domain in soup_dynamic.title.string.lower():
                            df.at[index, 'domain_in_title'] = 1.0

                    # --- 14. domain_with_copyright: 網站的版權資訊中是否包含域名 ---
                    df.at[index, 'domain_with_copyright'] = 0.0

                    # 檢查版權文本
                    copyright_text = soup_dynamic.find(text=re.compile(r'©|copyright', re.IGNORECASE))
                    if copyright_text and base_domain in copyright_text.lower():
                        df.at[index, 'domain_with_copyright'] = 1.0

                    # 檢查footer區域
                    footer_tags = soup_dynamic.find_all(['div', 'footer'], class_=re.compile(r'footer|copyright', re.IGNORECASE))
                    for footer in footer_tags:
                        if base_domain in footer.get_text().lower():
                            df.at[index, 'domain_with_copyright'] = 1.0
                            break

                    # 檢查所有包含版權相關文字的標籤
                    copyright_tags = soup_dynamic.find_all(text=re.compile(r'©|copyright|all rights reserved', re.IGNORECASE))
                    for tag in copyright_tags:
                        if base_domain in tag.lower():
                            df.at[index, 'domain_with_copyright'] = 1.0
                            break

                    df.at[index, 'feature_extracted'] = 1.0
                    print('✅ 資料處理完成')
                except Exception as e:
                    print(f"解析 URL '{url}' 的 HTML 時發生錯誤: {e}")
                    for col in html_feature_columns:
                        df.at[index, col] = 0.0
            else:
                print(f"未能獲取 URL '{url}' 的 HTML 內容。所有 HTML 特徵將為 0.0。")
            time.sleep(0.5)
            SAVE_INTERVAL = 300
            if (index + 1) % SAVE_INTERVAL == 0:
                print(f"--- 已處理 {index+1} 筆，正在儲存進度... ---")
                df.to_csv('phishing_dataset_expansion_forEmbeddingModule_html.csv', index=False)
        driver.quit()
        return df

    except KeyboardInterrupt:
        driver.quit()
        # df.to_csv('phishing_dataset_expansion_forEmbeddingModule_html.csv', index=False)
        print("\n🛑 KeyboardInterrupt (Ctrl+C) received by user.")

        return df


import signal

class TimeoutException(Exception):
    pass

def handler(signum, frame):
    raise TimeoutException()

def safe_process_url(driver, url, max_time=30):
    signal.signal(signal.SIGALRM, handler)
    signal.alarm(max_time)  # 設定最多處理 max_time 秒
    try:
        html_content, has_js_redirect = get_html_content(url, timeout=20, max_retries=1)
        dynamic_content = None
        if html_content:
            dynamic_content, state = _fetch_dynamic_content(driver, url)

        # 成功就清掉 alarm
        signal.alarm(0)
        return html_content, dynamic_content
    except TimeoutException:
        print(f"⏰ URL {url} 超過 {max_time} 秒，強制跳過")
        return None, False
    except Exception as e:
        print(f"❌ URL {url} 處理失敗: {e}")
        return None, False


# --- 範例使用 ---
if __name__ == "__main__":
    if os.path.exists('phishing_dataset_expansion_forEmbeddingModule_html.csv'):

        print(f"找到進度檔: {'phishing_dataset_expansion_forEmbeddingModule_html.csv'}。正在載入並繼續任務...")
        try:
            df= pd.read_csv('phishing_dataset_expansion_forEmbeddingModule_html.csv')
        except Exception as e:
            print(f"讀取時發生錯誤: {e}。")
            exit()
    else:
        print('error')
        exit()
    # 呼叫函式創建新特徵
    # 注意: 會自動先使用 requests，如果內容為空則使用 Selenium
    df = extract_html_features(df, url_column='url')

    # save_df(df_with_html_features, 'output_with_features_5000toEnd.csv')
    df.to_csv('phishing_dataset_expansion_forEmbeddingModule_html.csv', index=False)
    print('存檔成功')


找到進度檔: phishing_dataset_expansion_forEmbeddingModule_html.csv。正在載入並繼續任務...
開始設定DRIVER...


<IPython.core.display.Javascript object>

WebDriver 初始化完成。
正在處理第 2078 / 100000 筆 URL: http://www.antongas-fx.ru
[重試 1/1] Requests 獲取 http://www.antongas-fx.ru 時發生錯誤: HTTPConnectionPool(host='www.antongas-fx.ru', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7e383b49f470>: Failed to resolve 'www.antongas-fx.ru' ([Errno -2] Name or service not known)"))
未能獲取 URL 'http://www.antongas-fx.ru' 的 HTML 內容。所有 HTML 特徵將為 0.0。
正在處理第 2079 / 100000 筆 URL: https://securebusiness.github.io/bt-1/load.html
⚠️  requests 獲取的內容為空或過短，將嘗試使用 Selenium
未能獲取 URL 'https://securebusiness.github.io/bt-1/load.html' 的 HTML 內容。所有 HTML 特徵將為 0.0。


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'securebusiness.github.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


正在處理第 2080 / 100000 筆 URL: https://www.youlikehits.com
✅ 使用 requests 成功獲取 https://www.youlikehits.com 的內容


...

✅ 使用 requests 成功獲取 https://www.buyoutsinsider.com 的內容
✅ 網頁 https://www.buyoutsinsider.com 動態爬取完成...
✅ 資料處理完成
正在處理第 2086 / 100000 筆 URL: https://www.operating-system.org


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.operating-system.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ 使用 requests 成功獲取 https://www.operating-system.org 的內容
✅ 網頁 https://www.operating-system.org 動態爬取完成...
✅ 資料處理完成
正在處理第 2087 / 100000 筆 URL: https://www.historicalcharts.noaa.gov


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.historicalcharts.noaa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ 使用 requests 成功獲取 https://www.historicalcharts.noaa.gov 的內容
